In [ ]:
# Transformer编码器层核心代码（PyTorch示例）
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048):
        super().__init__()
        self.self_attn = MultiheadAttention(d_model, nhead)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, src, src_mask=None):
        # 自注意力计算
        src2 = self.self_attn(src, src, src, attn_mask=src_mask)[0]
        src = src + self.norm1(src2)  # 残差连接+归一化
        # 前馈网络
        src2 = self.linear2(F.relu(self.linear1(src)))
        src = src + self.norm2(src2)
        return 
def scaled_dot_product_attention(q, k, v, mask=None):
    matmul_qk = torch.matmul(q, k.transpose(-2, -1))
    dk = k.size(-1)
    scaled_attention_logits = matmul_qk / math.sqrt(dk)
    if mask: scaled_attention_logits += mask * -1e9
    attention_weights = F.softmax(scaled_attention_logits, dim=-1)
    return torch.matmul(attention_weights, v)

In [ ]:
#self_attention
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads=1):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size #输入向量的维度
        self.heads = heads
        self.head_dim = embed_size // heads

        self.to_qkv = nn.Linear(embed_size, 3 * embed_size)

    def forward(self, x):
        batch_size, seq_len, embed_size = x.shape
        #生成Q,K,V 通过一次线性变换分割成三部分
        qkv = self.to_qkv(x)
        q, k, v = torch.chunk(qkv, 3, dim=-1)  #分割为Q/K/V

        #计算缩放点积注意力
        scale = (self.head_dim) ** -0.5  #缩放因子根号d， 防止点积过大
        attention_scores =  torch.matmul(q, k.transpose(-2, -1)) * scale

        #应用softmax得到注意力权重
        attention_weights = F.softmax(attention_scores, dim=-1)

        out = torch.matmul(attention_weights, v)

        return out, attention_weights



In [ ]:
class multiattention(nn.Module):
    def __init__(self, embed_size,head=8):
        self.head = head
        self.embed_size = embed_size
        self.head_dim = embed_size // head

        self.to_qkv = nn.Linear(embed_size, 3*embed_size)
        self.to_out = nn.Linear(embed_size, embed_size)
    def forward(self,x):
        batch_size, seq_len, embed_size = x.shape()
        qkv = self.to_qkv(x).view(batch_size, seq_len, 3, self.head, self.head_dim)
        qkv = qkv.permute(2,0,3,1,4)
        q,k,v = qkv[0], qkv[1], qkv[2]

        attention_scores = torch.matmul(q,k.transpose(-2,-1)) * self.head_dim ** -0.5

        attention_weights = F.softmax(attention_scores, dim=-1)

        out = torch.matmul(attention_weights, v)

        out = out.permute(0,2,1,3).contiguous.view(batch_size, seq_len, embed_size)

        out = self.to_out(out)

    

SyntaxError: incomplete input (2631102471.py, line 10)

In [ ]:
class multiheadattention(nn.Module):
    def __init__(self, head, embed_size):
        super(multiattention(), self).__init__()
        self.embed_size = embed_size
        self.head = head
        self.head_dim = embed_size // head

        self.to_qkv = nn.Linear(embed_size,3 * embed_size)
        self.to_out = nn.Linear(embed_size,embed_size)
    def forward(self,x):
        batch_size, seq_len, embed_size = x.shape()
        qkv = self.to_qkv(x).view(batch_size, seq_len, 3, self.head, self.head_dim)
        qkv = qkv.permute(2,0,3,1,4)
        q,k,v = qkv[0], qkv[1], qkv[2]

        scale = self.head_dim ** -0.5
        attention_score = torch.matmul(q, k.transpose(-2,-1)) * scale
        attention_weight = F.softmax(attention_score, -1)

        out = torch.matmul(attention_weight, v)
        out = out.permute(0,2,1,3).contiguous().view(batch_size, seq_len, embed_size)
        out = self.to_out(out)

class GQA(nn.Module):
    def __init__(self, head, d_model, group):
        self.head = head
        self.d_model = d_model
        self.head_dim = d_model // head
        self.group = group

        self.Wq = nn.linear(d_model, d_model)
        self.Wk = nn.linear(d_model, d_model // group)
        self.Wv = nn.linear(d_model, d_model)
        self.out = nn.linear(d_model, d_model)

    def forward(self, x):
        batch_size, seq_len, d_model = x.shape()

        q = self.Wq(x).view(batch_size, -1, self.head, self.head_dim)
        k = self.Wq(x).view(batch_size, -1, self.head // self.group, self.head_dim)
        v = self.Wq(x).view(batch_size, -1, self.head // self.group, self.head_dim)

        scale = self.head_dim ** -0.5
        attn_score = torch.matmul(q,k.transpose(-2,-1)) * scale
        attn_weight = F.softmax(attn_score, -1)

        out = torch.matmul(attn_weight, v)

        out = out.permute(0,2,1,3).contiguous.view(batch_size, seq_len, d_model)

        out = self.out(out)

        return out

        




















In [ ]:
import torch.nn.functional as F
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module): # 类名改为驼峰式
    def __init__(self, head, embed_size):
        # 1. 修正super()的调用
        super().__init__() # 或者 super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.head = head
        self.head_dim = embed_size // head
        assert self.head_dim * head == embed_size, "embed_size 必须能被 head 整除"

        self.to_qkv = nn.Linear(embed_size, 3 * embed_size)
        self.to_out = nn.Linear(embed_size, embed_size)

    def forward(self, x):
        # 输入 x 的 shape: [batch_size, seq_len, embed_size]
        
        # 2. 修正 .shape() 为 .shape
        batch_size, seq_len, embed_size = x.shape

        # 线性变换并重塑
        # self.to_qkv(x) -> shape: [batch_size, seq_len, 3 * embed_size]
        qkv = self.to_qkv(x).view(batch_size, seq_len, 3, self.head, self.head_dim)
        # qkv shape: [batch_size, seq_len, 3, head, head_dim]

        # 维度重排，将 q, k, v 分离
        qkv = qkv.permute(2, 0, 3, 1, 4)
        # qkv shape: [3, batch_size, head, seq_len, head_dim]

        # 分离 q, k, v
        q, k, v = qkv[0], qkv[1], qkv[2]
        # q, k, v 的 shape 均为: [batch_size, head, seq_len, head_dim]

        # 计算注意力分数
        scale = self.head_dim ** -0.5
        attention_score = torch.matmul(q, k.transpose(-2, -1)) * scale
        # k.transpose(-2, -1) -> shape: [batch_size, head, head_dim, seq_len]
        # attention_score shape: [batch_size, head, seq_len, seq_len]

        attention_weight = F.softmax(attention_score, dim=-1)
        # attention_weight shape: [batch_size, head, seq_len, seq_len]

        # 注意力权重与 v 相乘
        out = torch.matmul(attention_weight, v)
        # out shape: [batch_size, head, seq_len, head_dim]

        # 合并多头
        out = out.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_len, embed_size)
        # out.permute(...) -> shape: [batch_size, seq_len, head, head_dim]
        # out.view(...) -> shape: [batch_size, seq_len, embed_size]

        out = self.to_out(out)
        # out shape: [batch_size, seq_len, embed_size]
        
        return out

In [ ]:
class GQA(nn.Module):
    def __init__(self, head, d_model, group):
        super().__init__() # 修正super()
        self.head = head
        self.d_model = d_model
        self.group = group
        self.kv_heads = head // group # K和V的头数
        self.head_dim = d_model // head
        
        assert head % group == 0, "head 必须能被 group 整除"

        # 1. 修正 nn.linear
        self.Wq = nn.Linear(d_model, d_model)
        # K和V的投影维度是 kv_heads * head_dim
        self.Wk = nn.Linear(d_model, self.kv_heads * self.head_dim)
        self.Wv = nn.Linear(d_model, self.kv_heads * self.head_dim)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, x):
        # 输入 x 的 shape: [batch_size, seq_len, d_model]
        
        # 2. 修正 .shape()
        batch_size, seq_len, d_model = x.shape

        # 投影并重塑 Q, K, V
        q = self.Wq(x).view(batch_size, seq_len, self.head, self.head_dim)
        # q shape: [batch_size, seq_len, head, head_dim]
        
        # 3. 修正 K, V 的投影
        k = self.Wk(x).view(batch_size, seq_len, self.kv_heads, self.head_dim)
        # k shape: [batch_size, seq_len, kv_heads, head_dim]
        
        v = self.Wv(x).view(batch_size, seq_len, self.kv_heads, self.head_dim)
        # v shape: [batch_size, seq_len, kv_heads, head_dim]

        # 维度重排以进行注意力计算
        q = q.transpose(1, 2) # -> [batch_size, head, seq_len, head_dim]
        k = k.transpose(1, 2) # -> [batch_size, kv_heads, seq_len, head_dim]
        v = v.transpose(1, 2) # -> [batch_size, kv_heads, seq_len, head_dim]

        # 5. GQA核心：重复K和V的头来匹配Q的头
        # [B, kv, S, D] -> [B, kv, 1, S, D] -> [B, kv, group, S, D] -> [B, head, S, D]
        k = k.unsqueeze(2).repeat(1, 1, self.group, 1, 1).view(batch_size, self.head, seq_len, self.head_dim)
        v = v.unsqueeze(2).repeat(1, 1, self.group, 1, 1).view(batch_size, self.head, seq_len, self.head_dim)
        # 现在 q, k, v 的 shape 均为: [batch_size, head, seq_len, head_dim]

        # 计算注意力（同MHA）
        scale = self.head_dim ** -0.5
        attn_score = torch.matmul(q, k.transpose(-2, -1)) * scale
        # attn_score shape: [batch_size, head, seq_len, seq_len]
        
        attn_weight = F.softmax(attn_score, dim=-1)
        # attn_weight shape: [batch_size, head, seq_len, seq_len]

        out = torch.matmul(attn_weight, v)
        # out shape: [batch_size, head, seq_len, head_dim]

        # 合并多头
        # 6. 修正 contiguous
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
        # out.transpose(...) -> shape: [batch_size, seq_len, head, head_dim]
        # out.view(...) -> shape: [batch_size, seq_len, d_model]

        out = self.out(out)
        # out shape: [batch_size, seq_len, d_model]

        return out

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LoRALayer(nn.Module):
    """
    LoRA (Low-Rank Adaptation) 层。
    
    该层通过在原始权重矩阵上叠加一个低秩更新矩阵 (ΔW = A * B) 来实现对预训练模型的微调。
    在训练期间，只有低秩矩阵 A 和 B 是可训练的，原始权重保持冻结。
    """
    def __init__(self, original_layer: nn.Linear, rank: int = 8, alpha: int = 16):
        """
        初始化 LoRA 层。

        参数:
            original_layer (nn.Linear): 要被适配的原始线性层。
            rank (int): 低秩矩阵的秩，即矩阵 A 和 B 的中间维度。
            alpha (int): 缩放因子，用于调整低秩更新的强度。
        """
        super().__init__()
        self.rank = rank
        self.alpha = alpha
        
        # 从原始层获取输入和输出维度
        self.in_dim = original_layer.in_features
        self.out_dim = original_layer.out_features

        # 复制原始权重，并将其设置为不可训练（冻结）
        self.original_weight = original_layer.weight.clone().detach()
        self.original_weight.requires_grad_(False)
        
        # 复制原始偏置（如果存在），并冻结
        self.original_bias = None
        if original_layer.bias is not None:
            self.original_bias = original_layer.bias.clone().detach()
            self.original_bias.requires_grad_(False)

        # 初始化低秩矩阵 A 和 B
        # A 使用高斯分布初始化，B 初始化为零
        self.A = nn.Parameter(torch.randn(self.in_dim, rank) * 0.02)
        self.B = nn.Parameter(torch.zeros(rank, self.out_dim))

        # 计算缩放因子
        self.scaling = alpha / rank

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        前向传播。

        参数:
            x (torch.Tensor): 输入张量。

        返回:
            torch.Tensor: 输出张量。
        """
        # 计算低秩更新矩阵 ΔW = A @ B * scaling
        delta_W = torch.matmul(self.A, self.B) * self.scaling
        
        # 将低秩更新叠加到原始权重上
        combined_weight = self.original_weight + delta_W
        
        # 使用组合后的权重执行线性变换
        return F.linear(x, combined_weight, self.original_bias)

In [ ]:
class Lora(nn.Module):
    def __init__(self, original_layer, rank,alpha):
        super().__init__()
        self.in_dim = original_layer.in_features
        self.out_dim = original_layer.out_features

        self.original_weights = original_layer.weights.clone().detach()
        self.original_weights.requires_grad_(False)

        self.A = nn.Parameter(torch.randn(self.in_dim, rank) * 0.02)
        self.B = nn.Parameter(torch.zeros(rank, self.out_dim))

        self.scale = alpha / rank

    def forward(self, x):
        delta_W = torch.matmul(self.A, self.B) * self.scale

        combined_weights = self.original_weights + delta_W

        return F.linear(x, combined_weights, self.original_bias)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QLoRALayer(nn.Module):
    """
    QLoRA (Quantized Low-Rank Adaptation) 层。

    该层在 LoRA 的基础上增加了量化步骤。它首先计算低秩更新矩阵 ΔW，
    然后对 ΔW 进行量化，最后将量化后的更新叠加到原始权重上。
    这进一步减少了微调过程中的内存占用。
    """
    def __init__(self, original_layer: nn.Linear, rank: int = 8, alpha: int = 16, quantization_bits: int = 8):
        """
        初始化 QLoRA 层。

        参数:
            original_layer (nn.Linear): 要被适配的原始线性层。
            rank (int): 低秩矩阵的秩。
            alpha (int): 缩放因子。
            quantization_bits (int): 用于量化低秩更新的位数。
        """
        super().__init__()
        self.rank = rank
        self.alpha = alpha
        self.quantization_bits = quantization_bits
        
        # 从原始层获取输入和输出维度
        self.in_dim = original_layer.in_features
        self.out_dim = original_layer.out_features

        # 复制原始权重和偏置，并将其设置为不可训练（冻结）
        self.original_weight = original_layer.weight.clone().detach()
        self.original_weight.requires_grad_(False)
        
        self.original_bias = None
        if original_layer.bias is not None:
            self.original_bias = original_layer.bias.clone().detach()
            self.original_bias.requires_grad_(False)

        # 初始化低秩矩阵 A 和 B
        self.A = nn.Parameter(torch.randn(self.in_dim, rank) * 0.02)
        self.B = nn.Parameter(torch.zeros(rank, self.out_dim))

        # 计算缩放因子
        self.scaling = alpha / rank

    def quantize(self, weight: torch.Tensor) -> torch.Tensor:
        """
        对权重进行一个简单的对称量化。

        参数:
            weight (torch.Tensor): 要量化的权重张量。

        返回:
            torch.Tensor: 量化后的权重张量。
        """
        # 计算量化范围
        q_range = 2 ** (self.quantization_bits - 1)
        
        # 找到权重的最大绝对值作为缩放基准
        scale = q_range / torch.max(torch.abs(weight))
        
        # 量化：缩放、四舍五入、裁剪范围
        quantized_weight = (weight * scale).round().clamp(-q_range, q_range - 1)
        
        # 反量化：将量化后的整数值转换回浮点数
        dequantized_weight = quantized_weight / scale
        return dequantized_weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        前向传播。

        参数:
            x (torch.Tensor): 输入张量。

        返回:
            torch.Tensor: 输出张量。
        """
        # 计算低秩更新矩阵 ΔW
        delta_W = torch.matmul(self.A, self.B) * self.scaling
        
        # 对低秩更新进行量化和反量化
        quantized_delta_W = self.quantize(delta_W)
        
        # 将量化后的更新叠加到原始权重上
        combined_weight = self.original_weight + quantized_delta_W
        
        # 使用组合后的权重执行线性变换
        return F.linear(x, combined_weight, self.original_bias)

In [ ]:
#MultiAttention
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        assert embed_size % heads == 0, #embed_size 必须能被 heads 整除
        self.heads_dim = embed_size // heads

        self.to_qkv = nn.Linear(embed_size, 3 * embed_size)
        self.to_out = nn.Linear(embed_size, embed_size)
    def forward(self, x):
        batch_size, seq_len, embed_size = x.shape

        qkv = self.to_qkv(x).view(batch_size, seq_len, 3, self.heads, self.heads_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # 调整维度顺序为 (3, batch_size, heads, seq_len, heads_dim)  #方便之后K矩阵的转置
        q, k, v = qkv[0], qkv[1], qkv[2]  # 分离为查询 q、键 k、值 v，每个形状为 (batch_size, heads, seq_len, heads_dim)
        #q：batch_size, heads, seq_len, heads_dim
        scale = self.heads_dim ** -0.5  # 计算缩放因子，防止点积过大，等于 1/sqrt(heads_dim)
        attention_scores = torch.matmul(q,k.transpose(-2, -1)) * scale
        # 计算注意力分数：q 和 k 的转置点积后乘以缩放因子
        # k.transpose(-2, -1) 将 k 的最后两个维度 (seq_len, heads_dim) 转置为 (heads_dim, seq_len)
        # 结果形状为 (batch_size, heads, seq_len, seq_len)

        attention_wights = F.softmax(attention_scores, dim=-1)  # 对注意力分数应用 softmax，得到权重
        # dim=-1 表示沿最后一个维度（序列长度）归一化，形状保持 (batch_size, heads, seq_len, seq_len)
        out = torch.matmul(attention_wights, v)
        # attention_weights 和 v 的矩阵乘法，结果形状为 (batch_size, heads, seq_len, heads_dim)
        print(out)
        #用于合并最后两个维度heads*heads_dim
        out = out.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_len, embed_size)
        #permute后，batch_size, seq_len, heads, heads_dim
        # contiguous() 确保内存连续，view 重塑为 (batch_size, seq_len, embed_size)
        print(out)
        #batch_size, seq_len, embed_size
        out = self.to_out(out)
        print(out)
        return out   
    

class GQA(nn.Module):
    def __init__(self, num_head, groups, k_v_heads, embed_size):
        super().__init__()
        self.head_dim = embed_size // num_head
        self.num_head = num_head
        self.groups = groups
        self.k_v_heads = k_v_heads
        self.embed_size = embed_size

        self.q_proj = nn.Linear(embed_size, embed_size)
        self.k_proj = nn.Linear(embed_size, k_v_heads * self.head_dim)
        self.v_proj = nn.Linear(embed_size, k_v_heads * self.head_dim)
        self.out_proj = nn.Linear(embed_size, embed_size)
    def forward(self, x):
        batch_size, seq_len, head_dim = x.shape
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)

        q = q.view(batch_size, seq_len, self.groups, self.num_head, head_dim)
        q = q.permute(0,2,3,1,4)
        k = k.view(batch_size, seq_len, self.num_head, head_dim)
        k = k.permute(0,2,3,1)
        v = v.view(batch_size, seq_len, self.num_head, head_dim)
        v = v.permute(0,2,1,3)

        scale = self.head_dim ** -0.5
        score = torch.matmul(q,k) * scale   # q @ k

        weight = F.softmax(score, dim=-1) # (batch, groups, num_head, seq_len, seq_len)

        out = torch.matmul(weight, v)

        out = out.permute(0, 3, 1, 2, 4)
        
        out = out.contiguous().view(batch_size, seq_len, -1)


        return self.out_proj(out)

class GQA(nn.Module):
    def __init__(self, num_head, group, kv_heads, embed_size):
        super().__init__()
        self.head_dim = embed_size // num_head
        self.group = group
        self.num_head = num_head
        self.embed_size = embed_size

        self.q_proj = nn.Linear(embed_size, embed_size)
        self.k_proj = nn.Linear(embed_size, kv_heads * self.head_dim)
        self.v_proj = nn.Linear(embed_size, kv_heads * self.head_dim)
        self.out_proj = nn.Linear(embed_size, embed_size)

    def forward(self, x):
        batch_size, seq_len, embed_size = x.shape()
        
        q = self.q_proj(x)
        q = q.view(batch_size, seq_len, self.group, self.num_head, self.head_dim)
        k = self.k_proj(x)
        k = k.view(batch_size, seq_len, self.num_head, self.head_dim)
        v = self.v_proj(x)
        v = v.view(batch_size, seq_len, self.num_head, self.head_dim)

        

        

# 输入形状: (batch_size, seq_len, embed_size)
# BN统计量在 (batch_size, seq_len) 上计算，对每个 embed_size 通道独立归一化
bn = nn.BatchNorm1d(embed_size)
x_bn = bn(x)  # 输出形状仍为 (batch_size, seq_len, embed_size)

# 输入形状: (batch_size, seq_len, embed_size)
# LN统计量在 embed_size 上计算，每个样本的每个位置独立归一化
ln = nn.LayerNorm(embed_size)
x_ln = ln(x)  # 输出形状仍为 (batch_size, seq_len, embed_size)





SyntaxError: invalid syntax (3180739669.py, line 11)

In [ ]:
"""
GRPO 最小实现 - 体现核心思想
只保留：组内优势计算 + PPO裁剪 + 重要性采样
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Dict

class GRPO:
    """
    GRPO核心实现
    
    核心公式：
    1. 组内优势：advantage = (reward - group_mean) / group_std
    2. PPO损失：L = -min(r*A, clip(r, 1-ε, 1+ε)*A)
       其中 r = exp(log_π_new - log_π_old) (重要性采样比率)
    """
    
    def __init__(
        self,
        policy_net: nn.Module,
        lr: float = 1e-4,
        epsilon: float = 0.2,    # PPO裁剪范围
        group_size: int = 4,     # 组大小
        num_epochs: int = 4      # 更新轮数
    ):
        self.policy = policy_net
        self.optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr)
        self.epsilon = epsilon
        self.group_size = group_size
        self.num_epochs = num_epochs
    
    def compute_group_advantages(self, rewards: torch.Tensor) -> torch.Tensor:
        """
        组内优势计算 (GRPO核心)
        
        输入: rewards [batch_size] - 每个样本的奖励
        输出: advantages [batch_size] - 组内归一化的优势
        
        示例:
        rewards = [1.0, 2.0, 3.0, 4.0,  10.0, 11.0, 12.0, 13.0]
                   └────组1────┘        └──────组2──────┘
        
        组1优势 ≈ [-1.34, -0.45, 0.45, 1.34]  # 组内标准化
        组2优势 ≈ [-1.34, -0.45, 0.45, 1.34]  # 组内标准化
        """
        batch_size = len(rewards)
        num_groups = batch_size // self.group_size
        
        # 重塑为 [num_groups, group_size]
        rewards_grouped = rewards[:num_groups * self.group_size].view(
            num_groups, self.group_size
        )
        
        # 组内标准化
        mean = rewards_grouped.mean(dim=1, keepdim=True)
        std = rewards_grouped.std(dim=1, keepdim=True) + 1e-8
        advantages = (rewards_grouped - mean) / std
        
        return advantages.view(-1)  # 展平为 [batch_size]
    
    def ppo_loss(
        self, 
        old_log_probs: torch.Tensor,  # 旧策略对数概率
        new_log_probs: torch.Tensor,  # 新策略对数概率
        advantages: torch.Tensor       # 优势函数
    ) -> torch.Tensor:
        """
        PPO裁剪损失
        
        ratio = π_new / π_old = exp(log π_new - log π_old)
        L = -E[min(ratio * A, clip(ratio, 1-ε, 1+ε) * A)]
        """
        # 重要性采样比率
        ratio = torch.exp(new_log_probs - old_log_probs)
        
        # 两个目标函数
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1 - self.epsilon, 1 + self.epsilon) * advantages
        
        # 取悲观估计（最小值）
        return -torch.min(surr1, surr2).mean()
    
    def train_step(
        self,
        sequences: torch.Tensor,      # [batch_size, seq_len]
        old_log_probs: torch.Tensor,  # [batch_size]
        rewards: torch.Tensor         # [batch_size]
    ) -> Dict[str, float]:
        """
        一步训练
        
        流程:
        1. 计算组内优势（不需要价值网络）
        2. 多轮PPO更新
        """
        # 步骤1: 组内优势
        advantages = self.compute_group_advantages(rewards)
        
        # 步骤2: 多轮更新
        total_loss = 0.0
        for _ in range(self.num_epochs):
            self.optimizer.zero_grad()
            
            # 获取新策略的对数概率
            new_log_probs = self.policy.get_log_probs(sequences)
            
            # PPO损失
            loss = self.ppo_loss(old_log_probs, new_log_probs, advantages)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 1.0)
            self.optimizer.step()
            
            total_loss += loss.item()
        
        return {
            'loss': total_loss / self.num_epochs,
            'reward_mean': rewards.mean().item()
        }


# ============================================================================
# 使用示例
# ============================================================================

class SimplePolicy(nn.Module):
    """简单策略网络（示例）"""
    def __init__(self, vocab_size=100, hidden=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Embedding(vocab_size, hidden),
            nn.LSTM(hidden, hidden, batch_first=True),
            nn.Linear(hidden, vocab_size)
        )
    
    def get_log_probs(self, sequences):
        """计算序列对数概率"""
        # 简化实现：返回随机值（实际应计算真实概率）
        return torch.randn(sequences.size(0))


if __name__ == "__main__":
    print("=" * 70)
    print("GRPO 最小实现演示")
    print("=" * 70)
    
    # 初始化
    policy = SimplePolicy()
    grpo = GRPO(policy, lr=1e-4, epsilon=0.2, group_size=4)
    
    # 模拟数据
    batch_size = 8
    seq_len = 10
    
    sequences = torch.randint(0, 100, (batch_size, seq_len))
    old_log_probs = torch.randn(batch_size)
    
    # 模拟奖励：组1低分，组2高分
    rewards = torch.tensor([
        2.0, 2.1, 2.2, 2.3,  # 组1（低质量）
        8.0, 8.1, 8.2, 8.3   # 组2（高质量）
    ])
    
    print("\n1. 原始奖励")
    print("-" * 70)
    print(f"组1: {rewards[:4].tolist()}")
    print(f"组2: {rewards[4:].tolist()}")
    
    print("\n2. 组内优势（GRPO核心）")
    print("-" * 70)
    advantages = grpo.compute_group_advantages(rewards)
    print(f"组1优势: {[f'{x:.2f}' for x in advantages[:4].tolist()]}")
    print(f"组2优势: {[f'{x:.2f}' for x in advantages[4:].tolist()]}")
    print("\n✅ 关键观察：")
    print("   - 两组优势分布相同（都是从负到正）")
    print("   - 组内相对排序被保留，绝对值被消除")
    
    print("\n3. 执行训练")
    print("-" * 70)
    metrics = grpo.train_step(sequences, old_log_probs, rewards)
    print(f"损失: {metrics['loss']:.4f}")
    print(f"平均奖励: {metrics['reward_mean']:.4f}")
    
    print("\n" + "=" * 70)
    print("💡 GRPO vs PPO 对比")
    print("=" * 70)
    print("""
    PPO优势计算:
      advantage = R + γV(s') - V(s)  # 需要价值网络V
      全局标准化: (A - mean_all) / std_all
    
    GRPO优势计算:
      advantage = (R - mean_group) / std_group  # 只需奖励R
      组内标准化: 每组独立归一化
    
    GRPO优势:
      ✅ 不需要价值网络（省内存）
      ✅ 避免价值估计误差
      ✅ 更稳定（组间差异被消除）
      ✅ 适合稀疏奖励任务
    """)
    
    print("=" * 70)

In [ ]:
class LR(nn.Module):
    def __init__(self, input_dim = 1):
        super(LR, self).__init__()
        self.linear = nn.Linear(input_dim, input_dim)
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [ ]:
# ...existing code...
@register_policy_loss("gspo")
def compute_policy_loss_gspo(
    old_log_prob: torch.Tensor,      # 旧策略下，每个 token 的对数概率，形状: [batch_size, response_length]
    log_prob: torch.Tensor,          # 新策略下，每个 token 的对数概率，形状: [batch_size, response_length]
    advantages: torch.Tensor,        # 每个 token 的优势值 A_i,t，形状: [batch_size, response_length]
    response_mask: torch.Tensor,     # 响应 token 的 mask（1 表示参与损失，0 表示忽略），形状: [batch_size, response_length]
    loss_agg_mode: str = "seq-mean-token-mean",  # 损失聚合方式，推荐使用 seq-mean-token-mean
    config: Optional[DictConfig | ActorConfig] = None,
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    计算 GSPO 的截断策略梯度目标以及相关指标。

    GSPO 核心思想：
      * 先在“序列级别”上定义一个重要性采样比率 s_i(θ)，再把它分配到每个 token。
      * 最终在 token 级别做类似 PPO 的裁剪，从而稳定训练。

    参数说明:
        old_log_prob (torch.Tensor):
            旧策略 π_old 对每个 token 的对数概率，形状 (batch_size, response_length)。
        log_prob (torch.Tensor):
            当前策略 π_θ 对每个 token 的对数概率，形状 (batch_size, response_length)。
        advantages (torch.Tensor):
            每个 token 的优势 A_i,t，形状 (batch_size, response_length)。
        response_mask (torch.Tensor):
            响应 token 掩码，1 表示该位置参与损失，0 表示不参与。
        loss_agg_mode (str, optional):
            用于 agg_loss 的聚合模式。GSPO 推荐 "seq-mean-token-mean"：
            先对一个序列的 token 做平均，再对 batch 中的所有序列做平均。
        config:
            包含裁剪系数等超参数的配置对象，需提供 clip_ratio 或 clip_ratio_{low,high}。

    返回:
        pg_loss:        策略梯度损失（标量）。
        pg_clipfrac:    被裁剪的比例（上/下界合在一起统计）。
        ppo_kl:         近似 KL (对数比率的负号加权平均)，用于监控策略更新步长。
        pg_clipfrac_lower: 为兼容接口预留的下界裁剪比例，这里固定为 0。
    """

    # 必须提供 config（里面包含裁剪范围等超参数）
    assert config is not None
    # 如果配置中提供了 clip_ratio_low / clip_ratio_high，则使用它们；
    # 否则退化为对称裁剪 [-clip_ratio, +clip_ratio]。
    clip_ratio_low = config.clip_ratio_low if config.clip_ratio_low is not None else config.clip_ratio
    clip_ratio_high = config.clip_ratio_high if config.clip_ratio_high is not None else config.clip_ratio

    # negative_approx_kl = log π_new - log π_old
    # 这是 token 级别的“近似负 KL 项”，后面会在序列级别做平均。
    negative_approx_kl = log_prob - old_log_prob

    # ------------------------------------------------------------------
    # 1. 计算“序列级别”的重要性比率 log s_i(θ)
    # ------------------------------------------------------------------
    # 先计算每个序列的长度（即有效 token 数量），用 response_mask 排除 padding。
    # clamp(min=1) 防止出现长度为 0 的情况导致除零。
    seq_lengths = torch.sum(response_mask, dim=-1).clamp(min=1)

    # 对每个序列，把 (log π_new - log π_old) 在有效 token 上求平均：
    # negative_approx_kl_seq[i] ≈ (1/|y_i|) * Σ_t (log π_new - log π_old)
    # 这对应论文里 log s_i(θ) 的近似形式。
    negative_approx_kl_seq = torch.sum(negative_approx_kl * response_mask, dim=-1) / seq_lengths

    # ------------------------------------------------------------------
    # 2. 把“序列级别”的比率分配到每个 token 上
    # ------------------------------------------------------------------
    # 理论上：
    #   s_i,t(θ) = sg[s_i(θ)] * π_θ(y_i,t|x,y_i,<t) / sg[π_θ(y_i,t|x,y_i,<t)]
    # 在 log 空间可写成：
    #   log s_i,t(θ) = sg[log s_i(θ)] + log_prob - sg[log_prob]
    #
    # 这里 negative_approx_kl_seq ≈ log s_i(θ)，
    # detach() 保证在反向传播时不会对 s_i(θ) 再求梯度（只让 token 级的 log_prob 参与优化）。
    log_seq_importance_ratio = (
        log_prob
        - log_prob.detach()
        + negative_approx_kl_seq.detach().unsqueeze(-1)  # [batch, 1] -> [batch, 1,] 广播到 [batch, T]
    )

    # 为防止 exp 溢出，对 log 比率做一个上界截断（例如 ≤ 10）
    log_seq_importance_ratio = torch.clamp(log_seq_importance_ratio, max=10.0)  # 数值稳定性

    # 从 log 空间还原到真实的比率：
    #   seq_importance_ratio = s_i,t(θ)
    seq_importance_ratio = torch.exp(log_seq_importance_ratio)

    # ------------------------------------------------------------------
    # 3. 类似 PPO 的裁剪策略梯度目标
    # ------------------------------------------------------------------
    # 未裁剪的策略梯度损失：-A * r
    pg_losses1 = -advantages * seq_importance_ratio

    # 裁剪后的比率 r_clip ∈ [1 - clip_ratio_low, 1 + clip_ratio_high]
    pg_losses2 = -advantages * torch.clamp(
        seq_importance_ratio,
        1 - clip_ratio_low,
        1 + clip_ratio_high,
    )

    # 按 PPO 方式取“悲观”目标：对每个位置取两者的最大值
    # （因为损失是负的，最大值对应更小的收益）
    pg_losses = torch.maximum(pg_losses1, pg_losses2)

    # ------------------------------------------------------------------
    # 4. 在序列级别聚合损失
    # ------------------------------------------------------------------
    # 使用通用的 agg_loss 函数对 pg_losses 做聚合：
    #   - 使用 response_mask 排除 padding / 非响应 token
    #   - 按 loss_agg_mode（如 seq-mean-token-mean）在序列和 batch 维度上做平均
    pg_loss = agg_loss(
        loss_mat=pg_losses,
        loss_mask=response_mask,
        loss_agg_mode=loss_agg_mode,
    )

    # ------------------------------------------------------------------
    # 5. 统计一些 PPO 相关的监控指标
    # ------------------------------------------------------------------
    # pg_clipfrac：有多少位置使用了裁剪后的目标（即 pg_losses2 > pg_losses1）
    pg_clipfrac = verl_F.masked_mean(
        torch.gt(pg_losses2, pg_losses1).float(),
        response_mask,
    )

    # 为了兼容其他接口，这里返回一个固定为 0 的“下界裁剪比例”
    pg_clipfrac_lower = torch.tensor(0.0, device=pg_loss.device)

    # 近似 KL：-E[log π_new - log π_old]，只在响应位置上取平均
    ppo_kl = verl_F.masked_mean(-negative_approx_kl, response_mask)

    return pg_loss, pg_clipfrac, ppo_kl, pg_clipfrac_lower
# ...existing code...

In [ ]:
import torch
import torch.nn as nn

class RoPE(nn.Module):
    """
    旋转位置编码（Rotary Positional Embedding, RoPE）的最小实现。

    核心思想：
    - 将最后一个特征维 head_dim 拆成两两一组（偶数/奇数索引），每对 (x_even, x_odd)
      视作二维向量，按位置相关的角度 θ 进行二维旋转：
          [x_even', x_odd'] = [x_even, x_odd] · [[cosθ, -sinθ],
                                                [sinθ,  cosθ]]
    - θ 的频率随维度而变：θ_t,i = t / base^(2i/d)，t 为 token 位置，i 为维度对的索引。

    约定的输入输出形状：
    - 期望 q, k 形状为 [batch, num_heads, seq_len, head_dim]，且 head_dim 为偶数。
      如你的张量为 [batch, seq_len, num_heads, head_dim]，可先 permute 到该形状：
          x = x.permute(0, 2, 1, 3)  # [B, S, H, D] -> [B, H, S, D]
    - forward 返回与输入同形状的张量。

    参数:
        dim (int): 每个头的维度 head_dim，必须为偶数。
        base (float): 频率基数，常用 10000.0（LLaMA 等也可用 10000）。
    """

    def __init__(self, dim: int, base: float = 10000.0):
        super().__init__()
        assert dim % 2 == 0, "RoPE 要求 head_dim 为偶数"
        self.dim = dim
        self.base = base

        # inv_freq[i] = 1 / base^(2i/d), i=0..(d/2-1)
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        # 注册为 buffer，参与设备/精度迁移，但不作为可训练参数
        self.register_buffer("inv_freq", inv_freq, persistent=False)

    @torch.no_grad()
    def _build_sin_cos(self, seq_len: int, device, dtype, offset: int = 0):
        """
        构造给定序列长度的 sin/cos 表（广播到 dim 维），用于与输入逐元素相乘。

        返回:
            cos: [seq_len, dim]，模式为 [c0, c0, c1, c1, ...]
            sin: [seq_len, dim]，模式为 [s0, s0, s1, s1, ...]
        """
        # 位置索引（可带 offset，用于 KV-Cache 增量解码）
        t = torch.arange(seq_len, device=device, dtype=self.inv_freq.dtype) + offset  # [seq_len]
        # 外积得到 [seq_len, dim//2] 的角度矩阵
        freqs = torch.outer(t, self.inv_freq)  # θ_{t,i}
        # 扩展到 dim：每个频率重复两次，对应偶/奇位
        cos = torch.cos(freqs).repeat_interleave(2, dim=-1).to(dtype=dtype)  # [seq_len, dim]
        sin = torch.sin(freqs).repeat_interleave(2, dim=-1).to(dtype=dtype)  # [seq_len, dim]
        return cos, sin

    @staticmethod
    def _rotate_half(x: torch.Tensor) -> torch.Tensor:
        """
        将最后一维拆成偶/奇两部分并做二维旋转所需的 (x_even, x_odd) -> (-x_odd, x_even) 变换。
        形状不变，适配广播。
        """
        x_even = x[..., ::2]  # 偶数位
        x_odd  = x[..., 1::2]  # 奇数位
        # 堆叠成 [.., dim//2, 2]，应用旋转所需的交换与取负，再还原到 [..., dim]
        x_rot = torch.stack((-x_odd, x_even), dim=-1).reshape_as(x)
        return x_rot

    def apply_rotary(self, x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
        """
        对张量 x 应用 RoPE。要求：
          - x 形状为 [B, H, S, D]（序列维在倒数第二维，特征维在最后一维）
          - cos/sin 形状为 [S, D]，会自动广播为 [1,1,S,D]
        """
        # 广播到 [B, H, S, D]
        cos = cos.unsqueeze(0).unsqueeze(0)  # [1,1,S,D]
        sin = sin.unsqueeze(0).unsqueeze(0)  # [1,1,S,D]
        # 旋转：x' = x * cos + rotate_half(x) * sin
        return x * cos + self._rotate_half(x) * sin

    def forward(self, q: torch.Tensor, k: torch.Tensor, offset: int = 0):
        """
        对 (q, k) 同步应用 RoPE。

        参数:
            q, k: [batch, num_heads, seq_len, head_dim]
            offset: 位置偏移（增量解码时，等于已缓存的序列长度）

        返回:
            q_rot, k_rot: 与输入同形状
        """
        assert q.shape == k.shape, "q/k 形状需一致"
        B, H, S, D = q.shape
        assert D == self.dim, f"最后一维 head_dim={D} 与初始化 dim={self.dim} 不一致"

        cos, sin = self._build_sin_cos(S, q.device, q.dtype, offset=offset)
        q_rot = self.apply_rotary(q, cos, sin)
        k_rot = self.apply_rotary(k, cos, sin)
        return q_rot, k_rot


# ========================
# 使用示例（可删除）
# ========================
if __name__ == "__main__":
    B, H, S, D = 2, 4, 8, 64  # batch, heads, seq_len, head_dim
    rope = RoPE(dim=D, base=10000.0)

    q = torch.randn(B, H, S, D, dtype=torch.float32)
    k = torch.randn(B, H, S, D, dtype=torch.float32)

    # 第一次整序列编码（offset=0）
    q_rot, k_rot = rope(q, k, offset=0)
    print(q_rot.shape, k_rot.shape)  # torch.Size([2, 4, 8, 64]) torch.Size([2, 4, 8, 64])

    # 增量解码时：假设已缓存长度为 S（offset=S），新来 1 个 token
    q_new = torch.randn(B, H, 1, D)
    k_new = torch.randn(B, H, 1, D)
    q_rot_new, k_rot_new = rope(q_new, k_new, offset=S)
    print(q_rot_new.shape, k_rot_new.shape)  # torch.Size([2, 4, 1, 64]) torch.Size([2, 4, 1, 64])

In [ ]:
import torch
import torch.nn as nn

class LogisticRegression:
    def __init__(self, input_dim, lr=0.01):
        # 手动初始化权重和偏置（对应线性层参数）
        self.W = torch.randn(input_dim, 1, requires_grad=False) * 0.01
        self.b = torch.zeros(1, requires_grad=False)
        self.lr = lr  # 学习率

    def forward(self, X):
        """前向传播：计算线性输出并应用Sigmoid"""
        linear = torch.mm(X, self.W) + self.b  # X.shape: (n_samples, input_dim)
        return torch.sigmoid(linear)  # 输出概率 P(y=1|x)

    def loss(self, y_pred, y_true):
        """计算二元交叉熵损失"""
        epsilon = 1e-8  # 防止log(0)
        return -torch.mean(y_true * torch.log(y_pred + epsilon) + 
                          (1 - y_true) * torch.log(1 - y_pred + epsilon))

    def manual_update(self, X, y):
        """手动计算梯度并更新参数（不依赖optimizer）"""
        # 前向传播
        y_pred = self.forward(X)
        loss = self.loss(y_pred, y)
        
        # 反向传播手动求导（链式法则）
        n_samples = X.size(0)
        
        # 计算损失对输出的梯度 dL/dy_pred
        dL_dy_pred = (y_pred - y) / n_samples  # shape: (n_samples, 1)
        
        # 计算梯度 dL/dW 和 dL/db
        dL_dW = torch.mm(X.t(), dL_dy_pred)    # X.T @ dL_dy_pred
        dL_db = torch.sum(dL_dy_pred, dim=0)   # 沿样本维度求和
        
        # 参数更新（梯度下降）
        self.W -= self.lr * dL_dW
        self.b -= self.lr * dL_db
        
        return loss.item()

    def predict(self, X, threshold=0.5):
        """预测类别（0或1）"""
        with torch.no_grad():  # 禁用梯度计算
            proba = self.forward(X)
            return (proba >= threshold).float()

In [ ]:
def auc_count(self,y_true,y_prob):
    sorted_nums = torch.argsort(y_prob, descending=False)
    y_prob = y_prob[sorted_nums]
    y_true = y_true[sorted_nums]

    pos_num = (y_true == 1).sum().item()
    neg_num = (y_true == 0).sum().item()
    
    cnt_cum = 0
    same_prob = None
    same_prob_cnt = 0

    for i, prob in enumerate(y_prob, start=1):
        if prob != same_prob:
            if same_prob_cnt > 0:
                avg_prob = same_prob_cnt / same_prob_cnt
                cnt_cum += avg_prob * same_prob_cnt

                same_prob_cnt = 0
                prob_sum = 0
            same_prob = prob
        if y_true[i-1] == 1:
            same_prob_cnt += 1 
            prob_sum += prob
        
    if same_prob_cnt>0:
        avg_prob = prob_sum / same_prob_cnt
        cnt_cum += avg_prob * same_prob_cnt

    auc = (cnt_cum - pos_num*(pos_num+1)/2)/(pos_num * neg_num)

    return auc 

    

SyntaxError: incomplete input (2445662913.py, line 9)

pos_probs = y_prob[y_true == 1]
neg_probs = y_prob[y_true == 0]
comparison = pos_probs[:, None] > neg_probs[None, :]
auc = (comparison.sum() + 0.5 * (pos_probs[:, None] == neg_probs[None, :]).sum()) / (pos_num * neg_num)

In [ ]:
#AUC计算代码 表示正样本预测概率大于负样本预测概率的概率，评估二分类模型对正负样本的排序能力，反映区分两类样本的效果

# 按预测概率升序排列，并同步调整真实标签顺序
def auc_count(self, y_prob, y_true):
    sorted_nums = torch.argsort(y_prob, descending=False)
    y_true_sorted = y_true[sorted_nums]
    y_prob_sorted = y_prob[sorted_nums]

    pos_num = (y_true == 1).sum().item()
    neg_num = (y_true == 0).sum().item()

    if pos_num == 0 or neg_num == 0:
        return 0.0
    # 计算每个正样本的累计排名（处理相同概率的情况）
    cum_rank = 0.0
    cur_prob = None
    same_prob_count = 0
    same_rank_sum = 0

    for idx, prob in enumerate(y_prob_sorted, start = 1):
        if prob != cur_prob:
            if same_prob_count > 0:
                avg_rank = same_rank_sum / same_prob_count
                cum_rank += avg_rank * same_prob_count
                
                same_prob_count = 0
                same_rank_sum = 0.0
            cur_prob = prob
        #只找正样本的排名
        if y_true_sorted[idx - 1] == 1:  
            same_prob_count += 1
            same_rank_sum += idx
    # 处理最后一组相同概率的情况
    if same_prob_count > 0:
        avg_rank = same_rank_sum/ same_prob_count
        cum_rank += avg_rank * same_prob_count
    # 应用AUC计算公式
    auc = (cum_rank - pos_num*(pos_num+1)/2)/(pos_num * neg_num)  
    return auc      


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class KLDivergence(nn.Module):
    """
    KL 散度（Kullback–Leibler Divergence）计算模块

    KL(P || Q) = sum_x P(x) * log(P(x) / Q(x))

    一般用途：
      - 度量两个概率分布 P 和 Q 之间的“差异/不一致度”
      - 变分自编码器(VAE)中约束隐变量分布 q(z|x) 接近先验 p(z)
      - 蒸馏（distillation）中让学生模型分布接近老师模型分布
      - RL 中约束新旧策略分布不要差太多（如 PPO/GSPO 中的 KL 监控）
    """

    def __init__(self, epsilon: float = 1e-8):
        """
        参数:
            epsilon: 防止出现 log(0) / 除零 的极小值
        """
        super().__init__()
        self.epsilon = epsilon

    def forward(self, P_logits: torch.Tensor, Q_logits: torch.Tensor) -> torch.Tensor:
        """
        输入是 logits（未归一化分数），内部转成概率后计算 KL(P || Q)。

        参数:
            P_logits: 作为“真实/目标”分布的 logits，形状 [..., num_classes]
            Q_logits: 作为“近似/预测”分布的 logits，形状 [..., num_classes]

        返回:
            标量 KL 损失（batch/样本平均）
        """
        # 将 logits 转成概率分布，并加上 epsilon 提高数值稳定性
        P = F.softmax(P_logits, dim=-1)
        Q = F.softmax(Q_logits, dim=-1)

        P = P + self.epsilon
        Q = Q + self.epsilon

        # 计算对数概率
        logP = torch.log(P)
        logQ = torch.log(Q)

        # KL(P||Q) = sum P * (logP - logQ)
        kl_elements = P * (logP - logQ)          # [..., num_classes]
        kl_per_sample = kl_elements.sum(dim=-1)  # [...] 每个样本的 KL

        # 返回所有样本的平均
        return kl_per_sample.mean()


class BinaryCrossEntropy(nn.Module):
    """
    二元交叉熵（Binary Cross Entropy, BCE）损失模块

    对应公式（单样本）:
      y ∈ {0,1}, p = σ(z) 为预测为正类的概率
      BCE(y, p) = -[ y * log(p) + (1 - y) * log(1 - p) ]

    一般用途：
      - 二分类任务（如垃圾邮件识别，点击率预估）
      - 每个输出独立做二分类的多标签任务（multi-label）
      - Logistic Regression / 线性模型+sigmoid 的标准损失
    """

    def __init__(self, epsilon: float = 1e-8):
        """
        参数:
            epsilon: 防止 log(0) 的极小值
        """
        super().__init__()
        self.epsilon = epsilon

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        参数:
            logits: 模型的原始输出（未过 sigmoid），形状 [batch_size, 1] 或 [batch_size]
            targets: 真实标签，取值 0 或 1，形状与 logits 可广播一致

        返回:
            标量 BCE 损失（batch 平均）
        """
        # 先通过 sigmoid 把 logits 映射到 (0,1) 概率区间
        probs = torch.sigmoid(logits)

        # 裁剪概率，避免 log(0)
        probs = torch.clamp(probs, self.epsilon, 1.0 - self.epsilon)

        # 按公式计算逐样本损失
        # loss_i = -[y_i * log(p_i) + (1 - y_i) * log(1 - p_i)]
        loss = -(
            targets * torch.log(probs) +
            (1.0 - targets) * torch.log(1.0 - probs)
        )

        # 对所有样本取平均，得到标量损失
        return loss.mean()

In [ ]:
def kl(self, P, Q, epsilon = 1e-8):
    P += epsilon
    Q += epsilon

    P = F.softmax(P,dim=-1)
    Q = F.softmax(Q,dim=-1)

    logP = torch.log(P)
    logQ = torch.log(Q)

    KL_elements = P *(logP - logQ)
    P/Q - logp - logQ - 1
    KL = KL_elements.sum(dim=-1)

    return KL.mean()

In [ ]:
def kl(P,Q,epsilon = 1e-8):
    P += epsilon
    Q += epsilon

    P = F.softmax(P,dim=-1)
    Q = F.softmax(Q, dim=-1)

    logP = torch.log(P)
    logQ = torch.log(Q)

    KL = P * (logP - logQ)
    KL = KL.sum(dim=-1)

    return KL.mean()


P: tensor([[0.0469, 0.0489, 0.2302, 0.0418, 0.0568, 0.0148, 0.0487, 0.2712, 0.2014,
         0.0392],
        [0.0436, 0.3497, 0.0228, 0.0051, 0.0119, 0.0270, 0.0196, 0.0320, 0.4057,
         0.0826],
        [0.1846, 0.1609, 0.1473, 0.0572, 0.2800, 0.0317, 0.0273, 0.0310, 0.0408,
         0.0393],
        [0.0768, 0.0925, 0.2772, 0.1169, 0.0447, 0.0567, 0.1478, 0.0679, 0.1017,
         0.0178]])
P1: tensor([[0.0469, 0.0489, 0.2302, 0.0418, 0.0568, 0.0148, 0.0487, 0.2712, 0.2014,
         0.0392],
        [0.0436, 0.3497, 0.0228, 0.0051, 0.0119, 0.0270, 0.0196, 0.0320, 0.4057,
         0.0826],
        [0.1846, 0.1609, 0.1473, 0.0572, 0.2800, 0.0317, 0.0273, 0.0310, 0.0408,
         0.0393],
        [0.0768, 0.0925, 0.2772, 0.1169, 0.0447, 0.0567, 0.1478, 0.0679, 0.1017,
         0.0178]])
tensor(1.0280)


In [ ]:
#KL散度
import torch
import torch.nn as nn
import torch.nn.functional as F

def kl_divergence_manual(P, Q, epsilon=1e-8):
    """手动实现KL散度计算"""
    # 添加极小值避免log(0)导致数值不稳定
    P = P + epsilon
    Q = Q + epsilon
    
    # 归一化处理（若输入不是概率分布）
    # 沿最后一个维度求和并保持维度，将张量转化为概率分布
    P = P / P.sum(dim=-1, keepdim=True)
    Q = Q / Q.sum(dim=-1, keepdim=True)
    
    # 计算对数概率（KL散度核心操作）
    logP = torch.log(P)
    logQ = torch.log(Q)
    
    # 逐元素计算KL散度：P * (logP - logQ)
    kl_elements = P * (logP - logQ)
    
    # 沿最后一个维度求和（对每个样本的概率分布求和）
    kl = kl_elements.sum(dim=-1)
    
    # 返回批次平均值
    return kl.mean()

def kl_divergence_pytorch(P_logits, Q_logits):
    """使用PyTorch内置函数计算KL散度"""
    # 对Q的logits应用log_softmax得到对数概率（符合KLDivLoss输入要求）
    log_Q = F.log_softmax(Q_logits, dim=-1)
    
    # 对P的logits应用softmax得到概率分布（符合KLDivLoss目标要求）
    P = F.softmax(P_logits, dim=-1)
    
    # 初始化KL散度损失函数，reduction='batchmean'表示按批次均值返回
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    
    # 计算KL散度（输入log_Q需在前，目标P在后）
    loss = kl_loss(log_Q, P)
    
    return loss

In [ ]:
# BERT模型核心结构（简化版）
class BERT(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers, num_heads):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, hidden_size)
        self.segment_emb = nn.Embedding(2, hidden_size)
        self.pos_emb = nn.Parameter(torch.randn(1, 512, hidden_size))
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(hidden_size, num_heads) 
            for _ in range(num_layers)
        ])
        
    def forward(self, input_ids, segment_ids):
        # 嵌入融合
        token_emb = self.embeddings(input_ids)
        seg_emb = self.segment_emb(segment_ids)
        pos_emb = self.pos_emb[:, :input_ids.size(1), :]
        x = token_emb + seg_emb + pos_emb
        # 编码器堆叠
        for layer in self.encoder_layers:
            x = layer(x)
        return x

In [ ]:
#MLM（Masked Language Model）：随机遮蔽15%的token（80%替换为[MASK]，10%随机替换，10%保留原词）
class MaskedLM(nn.Module):
    def __init__(self, hidden_size, vocab_size):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.decoder = nn.Linear(hidden_size, vocab_size)

    def forward(self, hidden_states, masked_positions):
        # 提取被遮蔽位置的隐藏状态
        batch_size, seq_len, dim = hidden_states.shape
        flat_positions = masked_positions.view(-1)
        selected = hidden_states.view(-1, dim)[flat_positions]
        # 解码预测
        x = F.gelu(self.dense(selected))
        x = self.layer_norm(x)
        logits = self.decoder(x)
        return logits

In [ ]:
#交叉熵损失  -P(x)logQ(x) P为真实分布，Q为预测分布
import torch

def cross_entropy_loss(logits, labels):
    """
    logits: 模型原始输出 (未归一化), shape: (N, C)
    labels: 真实标签（类别索引）, shape: (N,)
    """
    # 数值稳定性处理: 减去最大值防止指数爆炸
    logits = logits - torch.max(logits, dim=1, keepdim=True)[0]
    
    # 计算LogSoftmax (等价于 log(softmax(logits)))
    exp_logits = torch.exp(logits)
    log_probs = logits - torch.log(torch.sum(exp_logits, dim=1, keepdim=True))
    
    # 提取真实类别对应的对数概率
    n_samples = logits.shape[0]
    true_class_logprobs = log_probs[range(n_samples), labels]
    
    # 计算损失（平均损失）
    loss = -torch.mean(true_class_logprobs)
    return loss

In [ ]:
class Lora(nn.modules):
    def __init__(self, in_dim, out_dim, rank=8, alpha = 16):
        super().__init__
        self.rank = rank
        self.alpha = alpha

        self.A = nn.Parameter(torch.randn(in_dim, rank) * 0.02)
        self.B = nn.Parameter(torch.zeros(rank, out_dim))

        self.scalling = alpha / rank

    def forward(self, x):
        delta_W = torch.matmul(self.A, self.B) * self.scalling

        return x @ (self.original_weight + delta_W)

SyntaxError: incomplete input (984388660.py, line 1)

In [ ]:
#LoRA
import torch
import torch.nn as nn
import torch.nn.functional as F

class LoRALayer(nn.Module):
    def __init__(self, original_layer, rank=8, alpha=16):
        super().__init__()
        self.rank = rank
        self.alpha = alpha
        
        self.in_dim = original_layer.in_features
        self.out_dim = original_layer.out_features

        self.original_weight = original_layer.weight.clone().detach()  # 克隆原始权重并禁止梯度更新
        self.original_weight.requires_grad_(False)
        # 缩放因子：alpha / rank，用于平衡低秩更新的强度

        self.A = nn.Parameter(torch.randn(self.in_dim, rank) * 0.02)
        self.B = nn.Parameter(torch.zeros(rank, self.out_dim))

        self.scaling = alpha / rank

    def forward(self, x):
        # 计算低秩更新 ΔW = A @ B * scaling
        delta_W = torch.matmul(self.A, self.B) * self.scaling
        # 将更新量叠加到原始权重（需从外部传入原始权重）
        combined_weight = self.original_weight + delta_W
        return F.linear(x, combined_weight, self.original_layer.bias)
    
original_linear = nn.Linear(768, 512)

    # 创建LoRA适配层
lora_layer = LoRALayer(original_linear, rank=8, alpha=16)

    # 前向传播时直接替换使用
x = torch.randn(32, 768)  # 输入维度
output = lora_layer(x)    # 输出维度 (32, 512)


class Lora(nn.Module):
    def __init__(self, original_layer, rank, alpha):
        super().__init__()
        self.rank = rank
        self.alpha = alpha

        self.input_dim = original_layer.in_features
        self.out_dim = original_layer.out_features

        self.original_weight = original_layer.weight.clone().detach()
        self.original_weight.require_grad_(False)

        self.A = nn.Parameter(torch.randn(self.input_dim, rank) *0.02)
        self.B = nn.Parameter(torch.zeros(rank, self.out_dim))

        self.scaling = alpha / rank
    
    def forward(self, x):
        delta_W = torch.matmul(self.A, self.B) * self.scaling
        combined_weight = self.original_weight + delta_W
        return F.linear(x, combined_weight, self.original_weight.bias)
    


In [ ]:
# RMSNorm代码
import torch
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(hidden_size))  # 可学习参数γ

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps) #倒数平方根

    def forward(self, x):
        return self.weight * self._norm(x.float()).type_as(x)
    



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Expert(nn.Module):
    """专家网络模块"""
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model)
        )
    
    def forward(self, x):
        return self.net(x)

class MoELayer(nn.Module):
    """MoE核心层"""
    def __init__(self, d_model=768, num_experts=8, top_k=2):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        
        # 专家池初始化
        self.experts = nn.ModuleList([Expert(d_model, d_model*4) for _ in range(num_experts)])
        
        # 门控网络
        self.gate = nn.Linear(d_model, num_experts)
        
        # 负载均衡辅助损失系数
        self.aux_loss_coef = 0.01

    def forward(self, x):
        batch_size, seq_len, d_model = x.shape
        
        # 门控网络计算
        logits = self.gate(x)  # [B, S, E]
        probs = F.softmax(logits, dim=-1)
        
        # Top-K专家选择
        topk_probs, topk_indices = probs.topk(self.top_k, dim=-1)  # [B, S, K]
        topk_probs = topk_probs / topk_probs.sum(dim=-1, keepdim=True)
        
        # 专家计算与组合
        outputs = torch.zeros_like(x)   #创建形式相同的全0张量
        for i in range(self.num_experts):
            # 创建当前专家的mask
            expert_mask = (topk_indices == i)
            if expert_mask.any():
                # 当前专家处理的token索引
                batch_idx, seq_idx = torch.where(expert_mask)
                
                # 获取对应输入并计算专家输出
                expert_input = x[batch_idx, seq_idx]
                expert_output = self.experts[i](expert_input)
                
                # 加权累加结果
                weights = topk_probs[batch_idx, seq_idx, expert_mask[expert_mask]]
                outputs[batch_idx, seq_idx] += expert_output * weights.unsqueeze(-1)
        
        # 负载均衡损失计算
        expert_mask = F.one_hot(topk_indices, self.num_experts).float()
        expert_usage = expert_mask.mean(dim=0).mean(dim=0)
        aux_loss = (expert_usage.std() + 1e-6) * self.aux_loss_coef
        
        return outputs, aux_loss
    












SyntaxError: incomplete input (1838695233.py, line 112)